<a href="https://colab.research.google.com/github/aksesss/lab2/blob/master/VGG16%2Bcifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.applications.vgg16 import VGG16
from tensorflow.keras.datasets import cifar10
import keras
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [0]:
class cif10:
    _cursor = 0
    _i1 = 0

    def __init__(self):
        (self.X_train, self.y_train), (self.X_test, self.y_test) = cifar10.load_data()
        self.X_train = self.X_train.astype('float32')
        self.X_test = self.X_test.astype('float32')
        self.X_train = self.X_train / 255
        self.X_test = self.X_test / 255

        #self.X_train = self.X_train.reshape([-1, 3072]) #32*32*3
        #self.X_test = self.X_test.reshape([-1, 3072])

        ohe = OneHotEncoder()
        self.y_train = ohe.fit_transform(self.y_train).toarray()
        self.y_test = ohe.fit_transform(self.y_test).toarray()
    
    def next_batch_train(self, batch_size=50):
        res = [self.X_train[self._cursor : self._cursor + batch_size],
               self.y_train[self._cursor : self._cursor + batch_size]]
        self._cursor = self._cursor + batch_size
        if (self._cursor+batch_size > self.X_train.shape[0]):
            self._cursor = 0
        return res

In [0]:
data = cif10()

In [0]:
data.X_train.shape

(50000, 32, 32, 3)

In [0]:
vgg = VGG16(weights = 'imagenet' , input_shape = (32, 32, 3), include_top = False)

In [0]:
#for layer in vgg.layers:
#  layer.trainable = False

In [0]:
#model = keras.models.Sequential()

In [0]:
model = keras.models.Sequential()

#model.add(vgg)
for layer in vgg.layers:
  model.add(layer)

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu', name='hidden1'))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(256, activation='relu', name='hidden2'))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(10, activation='softmax', name='predictions'))

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 8, 8, 256)        

In [0]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

sgd = keras.optimizers.SGD(lr=0.001, momentum=0.9)


model.compile(loss='categorical_crossentropy',optimizer= sgd,metrics=['accuracy'])

es = EarlyStopping(patience=5, monitor='val_accuracy', mode='max')
mc = ModelCheckpoint('./weights.h5', monitor='val_accuracy', save_best_only=True, mode='max')


# initialize the number of epochs and batch size
EPOCHS = 100
BS = 32

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, 
                         zoom_range=0.15, 
                         width_shift_range=0.2, 
                         height_shift_range=0.2, 
                         shear_range=0.15,
                          horizontal_flip=True, 
                         fill_mode="nearest")
 

In [0]:
# train the model
history = model.fit_generator(aug.flow(data.X_train,data.y_train, batch_size=BS),
                              validation_data = (data.X_test, data.y_test), 
                              steps_per_epoch=len(data.X_train))

Epoch 1/1
  545/50000 [..............................] - ETA: 29:40 - loss: 1.7684 - accuracy: 0.3494

KeyboardInterrupt: ignored

In [0]:
#history = model.fit(x=data.X_train, y=data.y_train, batch_size=200, epochs=40, verbose=1, validation_split=0.1, shuffle=True)

In [0]:
model.score(X_train, y_train)

AttributeError: ignored

In [0]:
#------------------------------------------------------------------------------
# VGG16 ON CIFAR_10
#------------------------------------------------------------------------------
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow.keras as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from keras.utils.np_utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score


#------------------------------------------------------------------------------
# Using VGG16 model, with weights pre-trained on ImageNet.
#------------------------------------------------------------------------------

vgg16_model = VGG16(weights='imagenet',
                    include_top=False, 
                    classes=10,
                    input_shape=(32,32,3)# input: 32x32 images with 3 channels -> (32, 32, 3) tensors.
                   )

#Define the sequential model and add th VGG's layers to it
model = Sequential()
#for layer in vgg16_model.layers:
#    layer.trainable = False
#    model.add(layer)
model.add(vgg16_model)

#------------------------------------------------------------------------------
# Adding hiddens  and output layer to our model
#------------------------------------------------------------------------------

from tensorflow.keras.layers import Dense, Flatten, Dropout
model.add(Flatten())
model.add(Dense(512, activation='relu', name='hidden1'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu', name='hidden2'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax', name='predictions'))

model.summary()


#------------------------------------------------------------------------------
#  Loading CIFAR10 data
#------------------------------------------------------------------------------

(X_train, y_train), (X_test, y_test) = k.datasets.cifar10.load_data()

print("******************")
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
 
# Convert class vectors to binary class matrices using one hot encoding
y_train_ohe = to_categorical(y_train, num_classes = 10)
y_test_ohe = to_categorical(y_test, num_classes = 10)

# Data normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train  /= 255
X_test /= 255

print("******************")
print(X_train.shape)
print(y_train_ohe.shape)
print(X_test.shape)
print(y_test_ohe.shape)


#------------------------------------------------------------------------------
# TRAINING THE CNN ON THE TRAIN/VALIDATION DATA
#------------------------------------------------------------------------------

# initiate SGD optimizer
sgd = optimizers.SGD(lr=0.001, momentum=0.9)

# For a multi-class classification problem
model.compile(loss='categorical_crossentropy',optimizer= sgd,metrics=['accuracy'])


es = EarlyStopping(patience=5, monitor='val_accuracy', mode='max')
mc = ModelCheckpoint('./weights.h5', monitor='val_accuracy', save_best_only=True, mode='max')


# initialize the number of epochs and batch size
EPOCHS = 100
BS = 32

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
                        horizontal_flip=True, fill_mode="nearest")
 
# train the model
history = model.fit_generator(aug.flow(X_train,y_train_ohe, batch_size=BS),validation_data=(X_test,y_test_ohe), 
                              steps_per_epoch=len(X_train) // BS,epochs=EPOCHS,callbacks=[es,mc])

#We load the best weights saved by the ModelCheckpoint
model.load_weights('./weights.h5')


#------------------------------------------------------------------------------
# PREDICT AND EVALUATE THE CNN ON THE TEST DATA
#------------------------------------------------------------------------------
preds = model.predict(X_test)#on prédit le Test
score_test = accuracy_score( y_test, np.argmax(preds, axis=1) )
print (' LE SCORE DE TEST : ', score_test)


train_loss, train_score = model.evaluate(X_train, y_train_ohe)
test_loss, test_score = model.evaluate(X_test, y_test_ohe)
print("Train Loss:", train_loss)
print("Test Loss:", test_loss)
print("Train Score:", train_score)
print("Test Score:", test_score)

Using TensorFlow backend.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
hidden1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
hidden2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
predictions (Dense)          (None, 10)                2

KeyboardInterrupt: ignored